In [1]:
import json 
import pandas as pd
from datetime import datetime, date
import math

with open("data.json", "r") as read_file:
    data = json.load(read_file)

with open("cred-privado.json", "r") as read_file:
    cred = json.load(read_file)

df = pd.DataFrame.from_dict(data['data'] + cred['data'])

with open("inputs.json", "r") as read_file:
    inputs = json.load(read_file)
    
cdi = inputs["CDI"]
ipca = inputs["IPCA"]

target = 0.90

exclude= ["BANCO MASTER S/A"]

cols = ['nickName', 'fee', 'yield', 'years', 'days', 'available', 'quantityAvailable', 'guaranteeFGC']

df['exclude'] = df.nickName.str.contains('|'.join(exclude))
df = df.loc[df['exclude'] == False]
del df['exclude']


In [2]:
df['raw_yield'] = df['fee'].str.replace(" CDI", "")
df['raw_yield'] = df['raw_yield'].str.replace("CDI +", "", regex=False)
df['raw_yield'] = df['raw_yield'].str.replace(",", ".")
df['raw_yield'] = df['raw_yield'].str.replace("%", "")
df['raw_yield'] = df['raw_yield'].str.replace("DOLAR PTAX ", "")
df['raw_yield'] = df['raw_yield'].str.replace("IPC-A + ", "", regex=False)
df['raw_yield'] = df['raw_yield'].str.replace("+", "", regex=False)

df['raw_yield'] = df['raw_yield'].astype(float)

df.loc[df['fee'].str.contains('CDI +', regex=False), 'raw_yield'] = 100 + df['raw_yield']

df['maturityDate'] = pd.to_datetime(df['maturityDate'])
df['days'] = (df['maturityDate'] - datetime.now()).dt.days
df['years'] = df['days'] / 365
df['years'] = df['years'].apply(math.ceil)

df['taxes'] = 22.5
df.loc[df['days'] > 180, 'taxes'] = 20.0
df.loc[df['days'] > 360, 'taxes'] = 17.5
df.loc[df['days'] > 720, 'taxes'] = 15.0
df.loc[df['product'] == 'LCA', 'taxes'] = 0.0
df.loc[df['product'] == 'LCI', 'taxes'] = 0.0
df.loc[df['product'] == 'CRI', 'taxes'] = 0.0
df.loc[df['product'] == 'CRA', 'taxes'] = 0.0

df['yield'] = (1 - df['taxes']/100.0)*df['raw_yield']
df['minInvestment'] = df['minimumQuantityForApplication'] * df['puMinValue']
df['available'] = df['quantityAvailable'] * df['puMinValue']

df = df.sort_values(by=['years','yield'], ascending=False)

ValueError: could not convert string to float: 'DOLAR PTAX  2.00'

In [ ]:
def tax_tier(days):
    tax = 0.225
    if days > 720:
        tax = 0.150
    elif days > 360:
        tax = 0.175
    elif days > 180:
        tax = 0.200
    return tax

def date_to_days(_date):
    d = date.fromisoformat(_date)
    days = (d - date.today()).days
    return days

def tax_free_yield(_yield, _date):
    days = date_to_days(_date)
    tax = tax_tier(days)
    return _yield * (1 - tax)

def target_yields(_date):
    days = date_to_days(_date)
    tax = tax_tier(days)
    return [cdi / (1 - tax), 100/(1 - tax)]


def best_options(_data, _years):
    _d = _data.loc[_data['available'] > 4000]
    _d = _d.loc[_d['quantityAvailable'] > 10]
    return _d.loc[_d['years'] == _years].head(10)

In [ ]:
# Filtra os Pós-fixados pela taxa liquida de impostos
pos = df.loc[df['indexers']=='Pós-fixado']
pos = pos.loc[df['yield'] > 100 * target]

# Filtra os Pré-fixados pela taxa liquida de impostos
pre = df.loc[df['indexers']=='Pré-fixado']


# Filtra os indexados ao IPCA pela taxa real liquida de impostos
infl = df.loc[df['indexers']=='Inflação']
infl = infl.loc[df['yield'] > 5.5]

# Pós-Fixado


In [ ]:
d = []
for i in range(0,30):
    o = best_options(pos, i)[cols]
    if len(o) > 0:
        d.append(o)

pos_bo = pd.concat(d)
pos_bo

,nickName,fee,yield,years,days,available,quantityAvailable,guaranteeFGC
24,LCA China Construction Bank - JUL/2023,95% CDI,95.0000,1,364,1.925000e+06,1925,True
22,LCA SICREDI - JUL/2024,98% CDI,98.0000,2,729,1.934500e+07,19345,True
13,CDB AL5 BANK - JUL/2024,114% CDI,96.9000,2,729,1.844000e+06,1844,True
15,CDB LUSO BRASILEIRO - JUL/2024,114% CDI,96.9000,2,729,9.010000e+05,901,True
16,CDB PERNAMBUCANAS FINANCIADOR - JUL/2024,113% CDI,96.0500,2,729,9.908000e+06,9908,True
23,LCA China Construction Bank - JUL/2024,"95,5% CDI",95.5000,2,729,1.560000e+05,156,True
17,CDB MIDWAY - RIACHUELO S.A. - JUL/2024,"109,85% CDI",93.3725,2,729,1.996700e+07,19967,True
7,CDB PERNAMBUCANAS FINANCIADOR - JUL/2025,"118,25% CDI",100.5125,3,1094,9.912000e+06,9912,True
8,CDB AL5 BANK - JUL/2025,118% CDI,100.3000,3,1094,1.985000e+06,1985,True
11,CDB LUSO BRASILEIRO - JUL/2025,117% CDI,99.4500,3,1094,9.760000e+05,976,True


# Pré-Fixado


In [ ]:
d = []
for i in range(0,30):
    o = best_options(pre, i)[cols]
    if len(o) > 0:
        d.append(o)

pre_bo = pd.concat(d)
pre_bo

,nickName,fee,yield,years,days,available,quantityAvailable,guaranteeFGC
2,LCA SICREDI - OUT/2027,"13,15%",13.15,6,1930,1.958586e+07,19794,True
3,LCA SICREDI - OUT/2027,"13,15%",13.15,6,1930,6.750024e+05,686,True
0,LCA SICREDI - NOV/2029,"13,25%",13.25,8,2680,1.377459e+06,1404,True
1,LCA SICREDI - NOV/2029,"13,25%",13.25,8,2680,1.922783e+07,19468,True


# Inflação


In [ ]:
d = []
for i in range(0,30):
    o = best_options(infl, i)[cols]
    if len(o) > 0:
        d.append(o)

infl_bo = pd.concat(d)
infl_bo

,nickName,fee,yield,years,days,available,quantityAvailable,guaranteeFGC
4,CDB BANCO CNH CAPITAL S/A - JUN/2027,"IPC-A + 7,70%",6.5450,5,1799,9.123328e+07,104673,True
29,CRI CONSTRUTORA TENDA - ABR/2028,"IPC-A + 7,77%",7.7700,6,2105,8.786655e+05,840,False
46,CRA JBS - NOV/2027,"IPC-A + 5,94%",5.9400,6,1952,1.550650e+06,1408,False
28,DEB MOVIDA - JUN/2029,"IPC-A + 8,06%",6.8510,7,2529,1.165653e+06,1161,False
30,DEB UNIDAS - ABR/2029,"IPC-A + 7,53%",6.4005,7,2468,1.311709e+06,1296,False
47,CRA BRASKEM - DEZ/2028,"IPC-A + 5,83%",5.8300,7,2347,9.988793e+05,956,False
40,CRA JBS - ABR/2031,"IPC-A + 6,13%",6.1300,9,3198,1.576272e+06,1460,False
45,CRA JBS - NOV/2030,"IPC-A + 6,03%",6.0300,9,3050,1.096280e+06,997,False
36,DEB CICLUS AMBIENTAL - JAN/2031,"IPC-A + 6,78%",5.7630,9,3108,1.003929e+06,922,False
27,DEB MOVIDA - JUN/2032,"IPC-A + 8,31%",7.0635,10,3625,2.387399e+06,2373,False
